In [2]:
import pandas as pd
import numpy as np

import pymc3 as pm
%matplotlib inline

import matplotlib.pyplot as plt

In [3]:
%run gen_files_array.ipynb

In [4]:
%run functions.ipynb 

In [5]:
# 'files' -> array containing all files 
# test with SN000858 object
find_object('SN000858', files)

[8199]

In [5]:
filters = ['desg' , 'desi' , 'desr' , 'desz']

In [7]:
mp

<module 'multiprocessing' from '/home/felipematheus/anaconda3/lib/python3.7/multiprocessing/__init__.py'>

In [61]:
#Without error treatment
dfs, t = get_df(files[4044])
t = t.reshape(len(t),1)

x, y, yerr = get_xys(dfs['desg'])
x = x.reshape(len(x),1)

In [35]:
#with error treatment std_dev
dfs, t = get_df(files[8199])
t = t.reshape(len(t),1)

x, y, yerr = get_xys(cleaning_df(dfs['desg'], method = 'std_dev', clean_neg = True))
x = x.reshape(len(x),1)

In [8]:
#with error treatment percentage
dfs, t = get_df(files[8199])
t = t.reshape(len(t),1)

x, y, yerr = get_xys(cleaning_df(dfs['desg'], method = 'percentage', clean_neg = True, percentage = 0.4))
x = x.reshape(len(x),1)

# Drafts 

## https://docs.pymc.io/notebooks/getting_started.html

In [ ]:
basic_model = pm.Model()

with basic_model:
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=10, shape=2)
    sd = pm.HalfNormal('sd', sd=1)
    
    mu = alpha
    
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sd, observed=y)
    
map_estimate = pm.find_MAP(model=basic_model)

map_estimate

map_estimate = pm.find_MAP(model=basic_model, method='powell')

map_estimate

with basic_model:
    # draw 500 posterior samples
    trace = pm.sample(len(t))
    
with basic_model:

    # instantiate sampler
    step = pm.Slice()

    # draw 5000 posterior samples
    trace = pm.sample(len(t), step=step)

pm.traceplot(trace);

In [36]:
## plot the components
def prt():
    from bokeh.plotting import figure, show
    
    p = figure(title="SN000858 Filtro desg",
               x_axis_type='linear', plot_width=550, plot_height=350)
    p.yaxis.axis_label = 'Fluxo'
    p.xaxis.axis_label = 'Tempo (Dias)'
    
    # plot mean and 2σ region of total prediction
    upper = fit.mu_total + 1*fit.sd_total
    lower = fit.mu_total - 1*fit.sd_total
    band_x = np.append(fit.t.values, fit.t.values[::-1])
    band_y = np.append(lower, upper[::-1])
    
    # total fit
    p.line(fit.t, fit.mu_total,
           line_width=1, line_color="firebrick", legend="Total fit")
    p.patch(band_x, band_y,
            color="firebrick", alpha=0.6, line_color="white")
    
    
    p.circle(x.reshape(len(x)), y,
             color="black", legend="Observed data")
    
    show(p)

# ------------------

# Creating the GPs
#### For more info https://docs.pymc.io/notebooks/GP-MeansAndCovs.html

#### Exponentiated Quadratic - Cov fixed
$$k(x, x') = \mathrm{exp}\left[ -\frac{(x - x')^2}{2 \ell^2} \right]$$
### Bad

In [10]:
with pm.Model() as model:
    lengthscale = 4
    eta = 1.0
    cov = eta**2 * pm.gp.cov.ExpQuad(1, lengthscale)
    gp = pm.gp.Marginal(cov_func=cov)
    
    y_ = gp.marginal_likelihood("y", X=x, y=y, noise=yerr)

print("Predicting with gp ...")
mu, var = gp.predict(t, point=mp, diag=True)

fit = pd.DataFrame({"t": t.flatten(),
                    "mu_total": mu,
                    "sd_total": np.sqrt(var)})
print('Done')
prt()

/home/felipematheus/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x


Predicting with gp ...


/home/felipematheus/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/home/felipematheus/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/home/felipematheus/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In th

Done


#### Combine two dimension or more
### Not used

In [113]:
#with pm.Model() as model:
#    x1, x2 = np.meshgrid(np.linspace(0,1,10), np.arange(1,4))
#    X2 = np.concatenate((x1.reshape((30,1)), x2.reshape((30,1))), axis=0)    
#    ls = np.array([3.2, 5.0])
#    
#    cov = pm.gp.cov.ExpQuad(input_dim=1, ls=ls)
#
#    gp = pm.gp.Marginal(cov_func=cov)
#    
#    y_ = gp.marginal_likelihood("y", X=x, y=y, noise=yerr)
#    
#    print("Predicting with gp ...")
#    mu, var = gp.predict(t, point=mp, diag=True)
#    
#    fit = pd.DataFrame({"t": t.flatten(),
#                        "mu_total": mu,
#                        "sd_total": np.sqrt(var)})
#    print('Done')
#    prt()

#### Rational Quadratic - Cov fixed
### Bad

In [129]:
with pm.Model() as model:
    alpha = 0.1
    ls = 0.2
    tau = 0.3
    cov = tau * pm.gp.cov.RatQuad(1, ls, alpha)
    
    gp = pm.gp.Marginal(cov_func=cov)
    y_ = gp.marginal_likelihood("y", X=x, y=y, noise=yerr)
    
    print("Predicting with gp ...")
    mu, var = gp.predict(t, point=mp, diag=True)
    
    fit = pd.DataFrame({"t": t.flatten(),
                        "mu_total": mu,
                        "sd_total": np.sqrt(var)})
    print('Done')
    prt()

Predicting with gp ...
Done


#### Senoidal
### Not the case

In [ ]:
with pm.Model() as model:
    period = 50
    cov = pm.gp.cov.Cosine(1, period)
    gp = pm.gp.Marginal(cov_func=cov)
    y_ = gp.marginal_likelihood("y", X=x, y=y, noise=yerr)

    print("Predicting with gp ...")
    mu, var = gp.predict(t, point=mp, diag=True)
    
    fit = pd.DataFrame({"t": t.flatten(),
                        "mu_total": mu,
                        "sd_total": np.sqrt(var)})
    print('Done')
    prt()

#### Matern 5/2 - Cov fixed
### Bad

In [55]:
with pm.Model() as model:
    ls = pm.Gamma("l", alpha=4, beta=0.1)
    tau = 2 #pm.HalfCauchy("n", beta=2, testval=2.0)
    cov = tau * pm.gp.cov.Matern52(1, ls)
    
    gp = pm.gp.Marginal(cov_func=cov)
    y_ = gp.marginal_likelihood("y", X=x, y=y, noise=yerr)

In [56]:
print("Predicting with gp ...")
mu, var = gp.predict(t, point=mp, diag=True)

fit = pd.DataFrame({"t": t.flatten(),
                    "mu_total": mu,
                    "sd_total": np.sqrt(var)})
print('Done')
prt()

Predicting with gp ...
Done


/home/felipematheus/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x


In [68]:
with pm.Model() as model:
    ls = pm.Gamma("l", alpha=4, beta=0.1)
    tau = 2 #pm.HalfCauchy("n", beta=2, testval=2.0)
    cov = tau * pm.gp.cov.Matern52(1, ls)
    
    gp = pm.gp.Marginal(cov_func=cov)
    y_ = gp.marginal_likelihood("y", X=x, y=y, noise=yerr)

In [69]:
print("Predicting with gp ...")
mu, var = gp.predict(t, point=mp, diag=True)

fit = pd.DataFrame({"t": t.flatten(),
                    "mu_total": mu,
                    "sd_total": np.sqrt(var)})
print('Done')
prt()

Predicting with gp ...
Done


In [70]:
with pm.Model() as model:
    ls = pm.Gamma("l", alpha=4, beta=0.1)
    tau = pm.HalfCauchy("n", beta=2, testval=2.0)
    cov = tau*2 * pm.gp.cov.Matern52(1, ls=ls)
    
    gp = pm.gp.Marginal(cov_func=cov)
    y_ = gp.marginal_likelihood("y", X=x, y=y, noise=yerr)

In [71]:
print("Predicting with gp ...")
mu, var = gp.predict(t, point=mp, diag=True)

fit = pd.DataFrame({"t": t.flatten(),
                    "mu_total": mu,
                    "sd_total": np.sqrt(var)})
print('Done')
prt()

Predicting with gp ...
Done


/home/felipematheus/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x


In [25]:
#matern32

In [118]:
with pm.Model() as model:
    ls = pm.Gamma("l", alpha=4, beta=0.1)
    tau = pm.HalfCauchy("n", beta=2, testval=2.0)
    cov = tau*500 * pm.gp.cov.Matern32(1, ls)
    
    gp = pm.gp.Marginal(cov_func=cov)
    y_ = gp.marginal_likelihood("y", X=x, y=y, noise=yerr)

In [119]:
print("Predicting with gp ...")
mu, var = gp.predict(t, point=mp, diag=True)

fit = pd.DataFrame({"t": t.flatten(),
                    "mu_total": mu,
                    "sd_total": np.sqrt(var)})
print('Done')

Predicting with gp ...
Done


In [120]:
from bokeh.plotting import figure, show

p = figure(title="SN000858 Filtro desg",
           x_axis_type='linear', plot_width=550, plot_height=350)
p.yaxis.axis_label = 'Fluxo'
p.xaxis.axis_label = 'Tempo (Dias)'

# plot mean and 2σ region of total prediction
upper = fit.mu_total + 1*fit.sd_total
lower = fit.mu_total - 1*fit.sd_total
band_x = np.append(fit.t.values, fit.t.values[::-1])
band_y = np.append(lower, upper[::-1])

# total fit
p.line(fit.t, fit.mu_total,
       line_width=1, line_color="firebrick", legend="Total fit")
p.patch(band_x, band_y,
        color="firebrick", alpha=0.6, line_color="white")


p.circle(x.reshape(len(x)), y,
         color="black", legend="Observed data")

show(p)

In [113]:
#Modelo que deu certo Exp Quadratic

In [62]:
with pm.Model() as model:
    n = pm.HalfCauchy("n", beta=2, testval=2.0)
    l = pm.Gamma("l", alpha=4, beta=0.1)
    cov = n**3 * pm.gp.cov.ExpQuad(1, l)
    gp = pm.gp.Marginal(cov_func=cov)
    y_ = gp.marginal_likelihood("y", X=x, y=y, noise=yerr)
    
    #mp = pm.find_MAP(include_transformed=True)

/home/felipematheus/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x


In [63]:
print("Predicting with gp ...")
mu, var = gp.predict(t, point=mp, diag=True)

fit = pd.DataFrame({"t": t.flatten(),
                    "mu_total": mu,
                    "sd_total": np.sqrt(var)})
print('Done')

Predicting with gp ...
Done
